# Úkol č. 4 - regrese
**Deadline úkolu je uveden na [course pages](https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html).**

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je soubor `LifeExpectancyData.csv` na course pages (originál zde: https://www.kaggle.com/kumarajarshi/life-expectancy-who).
  * Popis datasetu najdete na uvedené stránce s originálem datasetu.
  * Cílová (vysvětlovaná) proměnná se jmenuje `Life expectancy `.
  

## Pokyny k vypracování
Body zadání, za jejichž (poctivé) vypracování získáte 12 bodů:

  1. Odeberte z dat body u kterých neznáte vysvětlovanou proměnnou.
  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat. Na jeho základě adekvátně reagujte na problematické věci v datech (chybějící hodnoty, atd.).
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte tento Jupyter Notebook.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer

from sklearn.model_selection import train_test_split,ParameterGrid, cross_val_score

from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

import matplotlib
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import plotly.express as px

%matplotlib inline

np.set_printoptions(precision=5, suppress=True)  # suppress scientific float notation (so 0.000 is printed as 0.

random_seed = 727

In [2]:
def simplePreprocessing(data):
    data.rename(columns={'Life expectancy ':'Life expectancy', 
    'Measles ' : 'Measles',
    ' BMI ' : 'BMI',
    'under-five deaths ' : 'under-five deaths ',
    'Diphtheria ' : 'Diphtheria',
    ' HIV/AIDS' : 'HIV/AIDS',
    ' thinness  1-19 years' : 'thinness  1-19 years',
    ' thinness 5-9 years' : 'thinness 5-9 years'
    }, inplace=True)
    
    #removes datapoints with missing target
    data.drop(data[data['Life expectancy'].isna()].index, inplace=True)
    
    data = data.drop(columns=['Country', 'Year'])
    
    data.rename(columns={'Status' : 'Developed'}, inplace=True)
    
    data['Developed'] = data['Developed'].apply(lambda x: 1 if x == 'Developed' else 0)
    
    return data

In [3]:
df = pd.read_csv('LifeExpectancyData.csv')
df = simplePreprocessing(df)
display(df)
display(df.shape)

,Developed,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,0,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,0,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,0,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,0,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,0,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0,44.3,723.0,27,4.36,0.000000,68.0,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,0,44.5,715.0,26,4.06,0.000000,7.0,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,0,44.8,73.0,25,4.43,0.000000,73.0,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,0,45.3,686.0,25,1.72,0.000000,76.0,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


(2928, 20)

In [4]:
X_rest, X_test, y_rest, y_test = train_test_split(
    df.drop(columns=['Life expectancy']), df['Life expectancy'], test_size=0.1, random_state=random_seed
)

X_train, X_val, y_train, y_val = train_test_split(
    X_rest, y_rest, test_size=0.2, random_state=random_seed
)

In [5]:
classifiers = [(LinearRegression, {}),
               (Ridge, {'alpha': range(20)}),
               (DecisionTreeRegressor, {'max_depth': range(1,31), 'criterion': ['mse', 'mae']}),
               (RandomForestRegressor, {'n_estimators': range(1, 51, 5), 'max_depth': range(1, 5)}),
               (AdaBoostRegressor,  {'n_estimators': range(1,51,5), 'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1]}),
               (KNeighborsRegressor, {'n_neighbors' : range(2, 50)})]

In [6]:
def transformDataFunctionCreation(train):
    scaler = StandardScaler()
    train = pd.DataFrame(scaler.fit_transform(train),index=train.index, columns=train.columns)
    
    imputer = KNNImputer(n_neighbors=5, weights='distance')
    train = pd.DataFrame(imputer.fit_transform(train),index=train.index, columns=train.columns)

    pca = PCA(n_components = 0.95)
    train = pd.DataFrame(pca.fit_transform(train))
    
    def transformFunction(x):
        x = pd.DataFrame(scaler.transform(x),index=x.index, columns=x.columns)
        x = pd.DataFrame( imputer.transform(x),index=x.index, columns=x.columns)
        x = pd.DataFrame( pca.transform(x))
        return x
        
    
    return train, transformFunction

#validates on fixed validation data
def GetError(method, param, X_train, Y_train, X_val, Y_val):
    X, FillAndTransform = transformDataFunctionCreation(X_train)

    dt = method(**param).fit(X, Y_train)
    pred = dt.predict(FillAndTransform(X_val))
    return mean_absolute_error(Y_val, pred)

#uses crossvalidation
def getCrossError(method, params, X, Y):
    pipeline = Pipeline([('scale', StandardScaler()),
                        ('fill', KNNImputer(n_neighbors=5, weights='distance')),
                        ('reduce_dim', PCA(n_components = 0.95)),
                        ('regressor', method(**params))
                        ])
    scores = cross_val_score(pipeline, X, Y, scoring='neg_mean_absolute_error')
    return -np.mean(scores)

bestCombo = []

for method, p in classifiers:
    param_comb = ParameterGrid(p)
    error = [getCrossError(method, param, X_rest, y_rest) for param in param_comb]
    best = np.argmin(error)
    
    print('using', method.__name__, param_comb[best])
    print(error[best])
    
    bestCombo.append((error[best], method, param_comb[best]))

using LinearRegression {}
3.0003449094779477
using Ridge {'alpha': 0}
3.000344909477948
using DecisionTreeRegressor {'max_depth': 12, 'criterion': 'mae'}
2.736318785578748
using RandomForestRegressor {'n_estimators': 46, 'max_depth': 4}
2.952475572250601
using AdaBoostRegressor {'n_estimators': 36, 'learning_rate': 1}
3.1349054827755354
using KNeighborsRegressor {'n_neighbors': 2}
1.8551423149905126


In [7]:
best = min(bestCombo, key=lambda x: x)
print('using', best[1].__name__, best[2])
error = GetError( method=best[1], param=best[2], X_train = X_rest, Y_train =y_rest,
                X_val = X_test, Y_val = y_test)
print('test MAE:', error)

using KNeighborsRegressor {'n_neighbors': 2}
test MAE: 1.772866894197952
